# 目標：有顧客清單和訂單，要找到從來沒有買過的顧客名稱
```python
table:customers
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
+-------------+---------+

Table: Orders
+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| customerId  | int  |
+-------------+------+

```



```sql
select a.name
from Customers a
left join Orders b
on a.id = b.customerId
where b.customerId is null
````

In [ ]:
# 第一種作法：merge()
import pandas as pd
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    merged = customers.merge(
        orders, 
        left_on = "id",
        right_on = "customerId",
        how = "left"
    )

    no_orders = merged[merged["customerId"].isna()]

    return no_orders[['name']].rename(columns={"name":"Customers"})

In [ ]:
# 第二種作法：isin()
import pandas as pd

def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    
    customers_with_orders = orders['customerId'].unique() # 去重找到唯一值的id

    no_orders = customers[~customers["id"].isin(customers_with_orders)] # id在order表裏（剛剛的customers_with_orders）但not in customers["id"]

    return no_orders[["name"]].rename(columns={"name": "Customers"})

In [ ]:
# 第三種作法：set差集

import pandas as pd

def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    # 先將兩者重點轉換為set
    all_customers = set(customers["id"])
    customers_with_orders = set(orders["customerId"])

    # 找出差集(那customers_without_orders是什麼？也是一個set嗎？)
    customers_without_orders =  all_customers - customers_with_orders # 找出沒有進到訂單系統的id?但這還是一個集合

    result = customers[customers["id"].isin(customers_without_orders)] # 萃取出customers裡面剛剛找到沒有的id的資料

    return result[["name"]].rename(columns={"name": "Customers"})